# Deploy an image classification model to the Triton Inference Server on local compute

This notebook shows you how to deploy a Densenet image classification model to the high-performance [Triton Inference Server](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/index.html) on local compute.

Please note that this Public Preview release is subject to the [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/).

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='default', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='azureml-examples')

## Download model

It's important that your model have this directory structure for Triton Inference Server to be able to load it. [Read more about the directory structure that Triton expects](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html).

In [2]:
import git
import os
import sys

from pathlib import Path

# get the root of the repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# Enables us to import helper functions as Python modules
path_to_insert = prefix.joinpath("code", "deployment", "triton").__str__()
if path_to_insert not in sys.path:
    sys.path.insert(1, path_to_insert)

from model_utils import download_triton_models, delete_triton_models


download_triton_models(prefix)

successfully downloaded model: densenet_onnx
successfully downloaded model: bidaf-9
successfully downloaded model: keiji_model


## Register model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.

In [3]:
from azureml.core.model import Model

model_path = prefix.joinpath("models")

model = Model.register(
    model_path=model_path,
    model_name="densenet-onnx-example",
    tags={"area": "Image classification", "type": "classification"},
    description="Image classification trained on Imagenet Dataset",
    workspace=ws,
)

print(model)

Registering model densenet-onnx-example
Model(workspace=Workspace.create(name='default', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='azureml-examples'), name=densenet-onnx-example, id=densenet-onnx-example:36, version=36, tags={'area': 'Image classification', 'type': 'classification'}, properties={})


## Deploy webservice

In this case we deploy to the local compute, but for other options, see [our documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where?tabs=azcli)


In [4]:
from azureml.core.webservice import LocalWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from random import randint

service_name = "triton-densenet-onnx-local" + str(randint(10000, 99999))
env = Environment.get(ws, "AzureML-Triton").clone("triton-example")

for pip_package in ["pillow"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(
    # this entry script is where we dispatch a call to the Triton server
    entry_script="score_densenet.py",
    source_directory=prefix.joinpath("code", "deployment", "triton"),
    environment=env,
)

config = LocalWebservice.deploy_configuration(port=6789)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=config,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 0e14976437204610b0f33e3f974544ac.azurecr.io
Logging into Docker registry 0e14976437204610b0f33e3f974544ac.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 0e14976437204610b0f33e3f974544ac.azurecr.io/azureml/azureml_84a593724077248bb83b9e6a297a2562
 ---> 3ba78c3888f6
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> f77d30488572
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjY1NjA1NzVkLWZhMDYtNGU3ZC05NWZiLWY5NjJlNzRlZmQ3YSIsInJlc291cmNlR3JvdXBOYW1lIjoiYXp1cmVtbC1leGFtcGxlcyIsImFjY291bnROYW1lIjoiZGVmYXVsdCIsIndvcmtzcGFjZUlkIjoiMGUxNDk3NjQtMzcyMC00NjEwLWIwZjMtM2UzZjk3NDU0NGFjIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in fdd2c2324354
/bin/bash: /azureml-envs/azureml_8e0ba8a051f495f86bcb9636c023c52d/lib/libtinfo.so.5: no version information available (requi

ERROR - Docker container start has failed, the port you are attempting to use is already in use:
500 Server Error: Internal Server Error ("driver failed programming external connectivity on endpoint wizardly_goldstine (35b4f19a4d742a3b81567def388091b4acb6612b674553048d79066d91a425a8): Bind for 127.0.0.1:6789 failed: port is already allocated")



Starting Docker container...


WebserviceException: WebserviceException:
	Message: Docker container start has failed, the port you are attempting to use is already in use:
500 Server Error: Internal Server Error ("driver failed programming external connectivity on endpoint wizardly_goldstine (35b4f19a4d742a3b81567def388091b4acb6612b674553048d79066d91a425a8): Bind for 127.0.0.1:6789 failed: port is already allocated")
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Docker container start has failed, the port you are attempting to use is already in use:\n500 Server Error: Internal Server Error (\"driver failed programming external connectivity on endpoint wizardly_goldstine (35b4f19a4d742a3b81567def388091b4acb6612b674553048d79066d91a425a8): Bind for 127.0.0.1:6789 failed: port is already allocated\")"
    }
}

## Test the webservice

In [ ]:
import requests

headers = {"Content-Type": "application/octet-stream"}

data_file = prefix.joinpath("data", "raw", "images", "peacock.jpg")
test_sample = open(data_file, "rb").read()
resp = requests.post(service.scoring_uri, data=test_sample, headers=headers)
print(resp.text)

## Delete the webservice and the downloaded model

In [ ]:
service.delete()
delete_triton_models(prefix)

# Next steps

Try changing the deployment configuration to [deploy to Azure Kubernetes Service](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-azure-kubernetes-service?tabs=python) for higher availability and better scalability.